1. Racer
Extend example project from lecture and finish following tutorials:

https://coderslegacy.com/python/python-pygame-tutorial/
https://coderslegacy.com/python/pygame-tutorial-part-2/
https://coderslegacy.com/python/pygame-tutorial-part-3/
Extra tasks to the given tutorial:

1) Adding randomly appearing coins on the road.
2) Showing the number of collected coins in the top right corner.
3) Comment your code.

In [3]:
# Import necessary libraries
import pygame, sys
from pygame.locals import *
import random, time

# Initialize pygame
pygame.init()

# Set up game constants
FPS = 60  # Frames per second
FramePerSec = pygame.time.Clock()  # Clock object to control frame rate

# Color definitions
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Game screen dimensions
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 3  # Initial game speed
SCORE = 0  # Player score
COINS = 0  # Collected coins count

# Font setup
font = pygame.font.SysFont("Verdana", 20)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, BLACK)

# Load background image
background = pygame.image.load("AnimatedStreet.png")

# Initialize game screen
screen = pygame.display.set_mode((400, 600))
screen.fill(WHITE)
pygame.display.set_caption("Racer")

# Enemy car class
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Enemy.png")  # Load enemy image
        self.rect = self.image.get_rect()  # Get image rectangle
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)  # Random starting position

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)  # Move enemy down the screen
        if (self.rect.top > 600):  # If enemy goes off screen
            SCORE += 1  # Increase score
            self.rect.top = 0  # Reset to top
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)  # New random position

# Flags for speed increases
c1,c2,c3,c4,c5 = False, False, False, False, False

# Coin class
class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("coin.png")  # Load coin image
        self.image = pygame.transform.scale(self.image, (40, 40))  # Resize coin
        self.rect = self.image.get_rect()  # Get image rectangle
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), random.randint(40, SCREEN_HEIGHT - 40))  # Random position

    def move(self):
        global COINS
        global SPEED
        # Different coin values based on position
        if self.rect.bottom<SCREEN_HEIGHT//3:
            COINS += 3
        elif self.rect.bottom<SCREEN_HEIGHT//1.5:
            COINS += 2
        else:
            COINS += 1
            
        # Increase speed at certain coin thresholds
        global c1,c2,c3,c4,c5
        if not c1 and COINS>=10:
            SPEED+=1
            c1=True
        if not c2 and COINS>=20:
            SPEED+=1
            c2=True
        if not c3 and COINS>=30:
            SPEED+=1
            c3=True
        if not c4 and COINS>=40:
            SPEED+=1
            c4=True
        if not c5 and COINS>=50:
            SPEED+=1
            c5=True
            
        # Move coin to new random position
        self.rect.top = random.randint(40, SCREEN_WIDTH - 40)
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), random.randint(40, SCREEN_HEIGHT - 40))

# Player car class
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Player.png")  # Load player image
        self.rect = self.image.get_rect()  # Get image rectangle
        self.rect.center = (160, 520)  # Starting position

    def move(self):
        pressed_keys = pygame.key.get_pressed()  # Get keyboard input
        # Move player based on key presses with boundary checks
        if self.rect.left > 0:
            if pressed_keys[K_LEFT]:
                self.rect.move_ip(-5, 0)
        if self.rect.right < SCREEN_WIDTH:
            if pressed_keys[K_RIGHT]:
                self.rect.move_ip(5, 0)
        if self.rect.top > 0:
            if pressed_keys[K_UP]:
                self.rect.move_ip(0, -5)
        if self.rect.bottom < SCREEN_HEIGHT:
            if pressed_keys[K_DOWN]:
                self.rect.move_ip(0, 5)

# Create game objects
P1 = Player()  # Player
E1 = Enemy()   # Enemy
C1 = Coin()    # Coin

# Create sprite groups
enemies = pygame.sprite.Group()
enemies.add(E1)
coinss = pygame.sprite.Group()
coinss.add(C1)
all_sprites = pygame.sprite.Group()
all_sprites.add(P1)
all_sprites.add(E1)
all_sprites.add(C1)

# Event to increase speed over time
INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 1000)

# Game over screen function
def game_over_screen():
    screen.fill(RED)
    screen.blit(game_over, (30, 250))
    pygame.display.update()

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == KEYDOWN:
                if event.key == K_SPACE: 
                    return True  # Restart game
                elif event.key == K_ESCAPE:
                    return False  # Quit game

# Handle crash event
def handle_crash():
    time.sleep(2)  # Pause before showing game over
    return game_over_screen()  # Show game over screen

# Background scrolling position
background_y = 0  

# Main game loop
while True:
    # Event handling
    for event in pygame.event.get():
        if event.type == INC_SPEED:  # Speed increase event
            SPEED += 0.1
        if event.type == QUIT:  # Quit event
            pygame.quit()
            sys.exit()

    # Check for collision with enemy
    if pygame.sprite.spritecollideany(P1, enemies):
        continue_game = handle_crash()
        if not continue_game:
            pygame.quit()
            sys.exit()

    # Background scrolling logic
    background_y = (background_y + SPEED) % background.get_height()
    screen.blit(background, (0, background_y))
    screen.blit(background, (0, background_y - background.get_height()))

    # Display score and coins
    scores = font_small.render(str(SCORE), True, BLACK)
    screen.blit(scores, (10, 10))

    coins = font_small.render(str(COINS), True, BLACK)
    screen.blit(coins, (370, 10))

    # Update and draw all sprites
    for entity in all_sprites:
        screen.blit(entity.image, entity.rect)

        # Handle coin collection
        if entity == C1:
            if pygame.sprite.spritecollideany(P1, coinss):
                entity.move()  # Move coin when collected
        else:
            entity.move()  # Move other entities

    # Move enemies
    for enemy in enemies:
        enemy.move()

    # Move coins and handle off-screen coins
    for coin in coinss:
        coin.rect.y += SPEED

        if coin.rect.top > SCREEN_HEIGHT:
            coin.rect.y = -coin.rect.height
            coin.rect.x = random.randint(40, SCREEN_WIDTH - 40)

    # Update display and maintain frame rate
    pygame.display.update()
    FramePerSec.tick(FPS)

SystemExit: 